<a href="https://colab.research.google.com/github/urstrulydeva/Fishes_identification/blob/main/Fishes_model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Activation, Add, Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Activation, Add, Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.query_conv = Conv2D(filters=input_shape[-1] // 8, kernel_size=1)
        self.key_conv = Conv2D(filters=input_shape[-1] // 8, kernel_size=1)
        self.value_conv = Conv2D(filters=input_shape[-1], kernel_size=1)
        self.gamma = self.add_weight(name='gamma', shape=[], initializer='zeros', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query_conv(inputs)
        key = self.key_conv(inputs)
        value = self.value_conv(inputs)

        attention = tf.matmul(tf.reshape(query, (tf.shape(query)[0], -1, tf.shape(query)[-1])),
                              tf.reshape(key, (tf.shape(key)[0], -1, tf.shape(key)[-1])), transpose_b=True)
        attention = tf.nn.softmax(attention)
        attention = tf.matmul(attention, tf.reshape(value, (tf.shape(value)[0], -1, tf.shape(value)[-1])))

        attention = tf.reshape(attention, tf.shape(inputs))
        return self.gamma * attention + inputs



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Dense, Reshape, Multiply


class ChannelAttention(Layer):
    def __init__(self, ratio=8, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        self.shared_dense = Dense(channels // self.ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)
        self.shared_dense_out = Dense(channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)
        self.global_avg_pool = GlobalAveragePooling2D()
        self.reshape = Reshape((1, 1, channels))

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = self.reshape(avg_pool)
        avg_pool = self.shared_dense(avg_pool)
        avg_pool = self.shared_dense_out(avg_pool)
        return Multiply()([inputs, avg_pool])

    def get_config(self):
        config = super(ChannelAttention, self).get_config()
        config.update({'ratio': self.ratio})
        return config


In [ ]:
def build_model(input_shape=(299, 299, 3), num_classes=6):
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)

    # Add self-attention mechanism after the final convolutional layer
    x = base_model.output
    x = SelfAttention()(x)
    x = ChannelAttention()(x)

    # Add global average pooling and output layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    return model



In [ ]:
model = build_model(input_shape=(299, 299, 3), num_classes=6)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 299, 299, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_830 (Conv2D)       │ (None, 149, 149, 32)   │            864 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_812   │ (None, 149, 149, 32)   │             96 │ conv2d_830[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_812            │ (None, 149, 149, 32)   │              0 │ batch_normalization_8… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_831 (Conv2D)       │ (None, 147, 147, 32)   │          9,216 │ activation_812[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_813   │ (None, 147, 147, 32)   │             96 │ conv2d_831[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_813            │ (None, 147, 147, 32)   │              0 │ batch_normalization_8… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_832 (Conv2D)       │ (None, 147, 147, 64)   │         18,432 │ activation_813[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_814   │ (None, 147, 147, 64)   │            192 │ conv2d_832[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_814            │ (None, 147, 147, 64)   │              0 │ batch_normalization_8… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 73, 73, 64)     │              0 │ activation_814[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_833 (Conv2D)       │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_815   │ (None, 73, 73, 80)     │            240 │ conv2d_833[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_815            │ (None, 73, 73, 80)     │              0 │ batch_normalization_8… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_834 (Conv2D)  

 Total params: 57,886,823 (220.82 MB)

 Trainable params: 57,826,279 (220.59 MB)

 Non-trainable params: 60,544 (236.50 KB)

In [ ]:
# Define the paths
train_dir = '/content/drive/MyDrive/Fishes_Detection/Trainingdata'
val_dir = '/content/drive/MyDrive/Fishes_Detection/Validationdata'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')


Found 1812 images belonging to 6 classes.
Found 132 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
checkpoint = ModelCheckpoint('model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=50,
    callbacks=[checkpoint, early_stopping])


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.4501 - loss: 1.6188 - val_accuracy: 0.8828 - val_loss: 0.9958
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 1.0000 - loss: 0.9825 - val_accuracy: 0.7500 - val_loss: 1.1005
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 626ms/step - accuracy: 0.9430 - loss: 0.8781 - val_accuracy: 0.9766 - val_loss: 0.4475
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.4326 - val_accuracy: 1.0000 - val_loss: 0.5499
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 634ms/step - accuracy: 0.9871 - loss: 0.3703 - val_accuracy: 0.9844 - val_loss: 0.2098
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 1.0000 - loss: 0.1329 - val_accuracy: 1.0000 - val_loss: 0.3215
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 641ms/step - accuracy: 0.9947 - loss: 0.1538 - val_accuracy: 0.9844 - val_loss: 0.1107
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 1.0000 - loss: 0.0945 - val_accuracy: 1.

In [ ]:
test_dir = '/content/drive/MyDrive/Fishes_Detection/Testingdata'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)  # Set shuffle=False to maintain the order of images


Found 254 images belonging to 6 classes.


In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('model.keras', custom_objects={'SelfAttention': SelfAttention,'ChannelAttention':ChannelAttention})



In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 257ms/step - accuracy: 0.9323 - loss: 0.2507
Test Loss: 0.15529735386371613
Test Accuracy: 0.9598214030265808
